In [37]:
import requests
import time
import json
from kafka import KafkaProducer


In [38]:
topic_name = 'volume_data'
bootstrap_servers = ['localhost:9092'] # адрес сервера Kafka

In [39]:

# Функция для получения списка всех бирж
def get_exchanges():
    url = "https://api.coingecko.com/api/v3/exchanges"
    response = requests.get(url)
    exchanges = response.json()
    exchange_list = [exchange['id'] for exchange in exchanges]
    return exchange_list


# Функция для получения информации об объемах торгов за один последний день на бирже
def get_volume_one_day(exchange_id):
    url = f"https://api.coingecko.com/api/v3/exchanges/{exchange_id}/volume_chart?days=1"
    response = requests.get(url)
    volume_one_day_data = response.json()
    return {exchange_id: volume_one_day_data}


# Генератор для сбора информации об объемах торгов за последний день на всех биржах
def generate_volume_data():
    exchange_list = get_exchanges()
    for exchange in exchange_list:
        time.sleep(7)
        volume_one_day_data = get_volume_one_day(exchange)
        yield json.dumps(volume_one_day_data).encode('utf-8')



    # Функция для отправки данных в Kafka
def send_data_kafka(topic_name, bootstrap_servers, data_generator):
    producer = KafkaProducer(bootstrap_servers=bootstrap_servers)
    for data in data_generator:
        producer.send(topic_name, data)
        time.sleep(1)  # добавляем задержку в 1 секунду между отправками, чтобы не перегрузить Kafka
    producer.flush()


In [ ]:
# Отправляем данные в Kafka

data_generator = generate_volume_data()
send_data_kafka(topic_name, bootstrap_servers, data_generator)

In [1]:
!pip install confluent-kafka


  Using cached confluent_kafka-2.1.0-cp310-cp310-macosx_11_0_arm64.whl (2.8 MB)
